Data Structure:

- 'website_name' : Name of the website. type: str
- 'category': news category ( such as political, society, economy etc.)
- 'title': title of the news. type: str, language: nepali/english
- 'authors': name of the author, type: str, example: कारोबार संवाददाता
- 'published_date': Date in Roman Calenader, example: December 17 2019, Tuesday => 17-12-2019
- 'content': content of news. must be greater than 30 words. type: str
- 'language', : language of content.
- 'url': url of that partiular news.

## Importing Libraries

Used Environment details :

**environ.yml** $\implies$

``` yaml
name: hmimalkhabar_processing
channels:
  - defaults
dependencies:
  - pip=23.2.1
  - python=3.11.4
  - pip:
      - jupyterlab==4.0.5
      - pandas==2.0.3
      - pybsdate==0.3.0
```

In [1]:
import numpy as np
import pandas as pd
from glob import glob
from pyBSDate import bsdate # Date conversion From Bikram Sambat(BS) to English Date(AD) and viceversa

In [2]:
# Lising json files inside the the directory
glob('*.json')

['HimalKhabar_news_collection.json']

## Reading the [HimalKhabar_news_collection.json](https://dagshub.com/Omdena/Nepal-Asia-Foundation/src/main/tasks/task-01-data-collection/Nepali_Newspapers/HimalKhabar_NEWS/HimalKhabar_news_collection.json) dataset

In [3]:
hmimalkhabar_df = pd.read_json('HimalKhabar_news_collection.json')
original_shape = hmimalkhabar_df.shape
print(f'Shape of the data set : {original_shape}\n')
hmimalkhabar_df.sample(4)

Shape of the data set : (21705, 6)



,Title,Category,Auther,Date,Content,URL
19194,देवी र रेणुबीच ७६ मतको अन्तर,समाचार,हिमालखबर,"शनिबार, २१ साउन, २०७४",भरतपुर महानगरपालिकाको जारी मतगणनामा एमाले उम्म...,https://www.himalkhabar.com/news/4824
11104,जिल्ला न्यायाधीश पदमा २९ जना माग हुँदा अन्तर्व...,समाचार,हिमालखबर,"आइतबार, ४ साउन, २०७७",न्याय सेवा आयोगले जिल्ला अदालतहरूमा रिक्त २९ प...,https://www.himalkhabar.com/news/116198
7974,कमानसिंह र हाइड्रोपावरको विवादमा नारायणकाजीलाई...,समाचार,हिमालखबर,"आइतबार, २८ मंसिर, २०७७",नेपाल कम्युनिष्ट पार्टी (नेकपा)का कार्यकारी अध...,https://www.himalkhabar.com/news/121322
18241,प्रदेश ३ः एमाले प्रदेश सभा संसदीय दलको नेतामा ...,समाचार,हिमालखबर,"शुक्रबार, २६ माघ, २०७४",प्रदेश ३ को नेकपा एमाले प्रदेश सभा संसदीय दलको...,https://www.himalkhabar.com/news/6680


## Cleaning the data

In [4]:
# Present column names

list(hmimalkhabar_df.columns)

['Title', 'Category', 'Auther', 'Date', 'Content', 'URL']

In [5]:
# Unifying column names and dropping unwanted columns

correcting_col_names = {'Title' : 'title', 'Category' : 'category',
                       'Auther' : 'authors', 'Content' : 'content', 
                       'URL' : 'url'}


hmimalkhabar_df = hmimalkhabar_df.rename(columns = correcting_col_names)
hmimalkhabar_df['website_name'] = 'HimalKhabar'
hmimalkhabar_df['language'] = 'Nepali'
hmimalkhabar_df.sample(3)

,title,category,authors,Date,content,url,website_name,language
13433,रौतहटको श्रीराम सुगर मिल सञ्चालनमा,समाचार,हिमालखबर,"बुधबार, २९ माघ, २०७६",चिनी मिल सञ्चालनका लागि आन्दोलन गर्दै आएको रौत...,https://www.himalkhabar.com/news/111542,HimalKhabar,Nepali
7032,"पुल बनाइरहेका मजदूर त्रिशूलीमा खसे, तीनको उद्ध...",समाचार,हिमालखबर,"आइतबार, १२ वैशाख, २०७८",त्रिशूलीमा पुल निर्माण गरिरहेका पाँच जना मजदूर...,https://www.himalkhabar.com/news/123016,HimalKhabar,Nepali
366,झूटो विवरणका आधारमा नागरिकता लिएको आरोपमा उपमे...,समाचार,वीरेन्द्र रमण,"बिहीबार, २१ असार, २०८०",झूटो विवरणका आधारमा नागरिकता लिएको आरोपमा सप्त...,https://www.himalkhabar.com/news/136861,HimalKhabar,Nepali


In [6]:
# Checking for missing value count

hmimalkhabar_df.isnull().sum()

title           0
category        0
authors         0
Date            0
content         0
url             0
website_name    0
language        0
dtype: int64

## Check for NEWS length

In [7]:
# Checking news contents that have atleast 30 words init.

def content_length_30(text):
    if len(text.split()) < 30:
        return np.nan
    return text.replace(' .', '')

hmimalkhabar_df.content = hmimalkhabar_df.content.apply(content_length_30)
mo_of_rejected_news = hmimalkhabar_df.content.isnull().sum()
print(f"Number of NEWS doesn't have  atleast 30 words = {mo_of_rejected_news}")

Number of NEWS doesn't have  atleast 30 words = 158


In [8]:
# Dropping rows where 'contents' doesn't have atleast 30 words

hmimalkhabar_df.dropna(subset = 'content', inplace = True)
hmimalkhabar_df.isnull().sum()

title           0
category        0
authors         0
Date            0
content         0
url             0
website_name    0
language        0
dtype: int64

## Check for dupllicates

In [9]:
# Checking number of duplicates

hmimalkhabar_df.duplicated().sum()

0

In [10]:
# Further Checking for duplicates

duplicates_df = hmimalkhabar_df[hmimalkhabar_df.duplicated(subset = 'url')]
rows_removed = duplicates_df.shape[0]
print(f'Number of ducplicate records in the dataset (based on url) = {rows_removed}\n')
duplicates_df.head()

Number of ducplicate records in the dataset (based on url) = 0



,title,category,authors,Date,content,url,website_name,language


Shows we don't have any redundancy in the dataset.

## Coverting BS calendar (Bikram Samwat) to Roman calendar

In [11]:
# https://calendar-nepali.com/about/Nepali-Calendar-Months.html

months_dict  = {'बैशाख': 1, 'वैशाख' : 1,
                'जेष्ठ': 2, 'जेठ' : 2,
                'असार': 3, 'आषाढ़' : 3,
                'साउन': 4, 'श्रावण' : 4, 'सावन' : 4,
                'भाद्र' : 5, 'भदौ': 5, 'भादो' : 5,
                'अशोज': 6, 'आश्विन' : 6, 'असोज' : 6, 'कुआर' : 6, 'क्वार' : 6,
                'कार्तिक': 7, 'कात्तिक' : 7,
                'मंसिर': 8, 'अगहन' : 8, 'मार्ग' : 8,
                'पुस': 9, 'पुष' : 9, 'पूस' : 9, 'पौष' : 9,
                'माघ': 10,
                'फाल्गुन': 11, 'फागुन' : 11,
                'चैत्र': 12, 'चैत' : 12}

In [12]:
# https://nepalilanguage.org/numbers/

number_dict = {'०': '0', '१': '1', '२': '2',
               '३': '3', '४': '4', '५': '5',
               '६': '6', '७': '7', '८': '8',
               '९': '9'}

In [13]:
# Removing ',' from date column
hmimalkhabar_df.Date = hmimalkhabar_df.Date.apply(lambda row : row.replace(',', ''))

# Printing some samples from Date column
hmimalkhabar_df.Date[4:15]

4     सोमबार २९ साउन २०८०
5     बुधबार ३१ साउन २०८०
6     बुधबार ३१ साउन २०८०
7          १५ मिनेट पहिले
8           १ घण्टा पहिले
9           ६ घण्टा पहिले
10          ८ घण्टा पहिले
11    बुधबार ३१ साउन २०८०
12          ४ घण्टा पहिले
13    बुधबार ३१ साउन २०८०
14    सोमबार २९ साउन २०८०
Name: Date, dtype: object

In [14]:
# Checking different formats count in the Date column

len_of_date = hmimalkhabar_df.Date.apply(lambda row : len(row.split()))
len_of_date.value_counts()

Date
4    21536
3       11
Name: count, dtype: int64

In [15]:
# Visualize dates that only have 3 parts in it

hmimalkhabar_df.Date[len_of_date.apply(lambda row : row ==3)]

7     १५ मिनेट पहिले
8      १ घण्टा पहिले
9      ६ घण्टा पहिले
10     ८ घण्टा पहिले
12     ४ घण्टा पहिले
16     ७ घण्टा पहिले
17    १२ घण्टा पहिले
21    ११ घण्टा पहिले
22    ११ घण्टा पहिले
23     ३ घण्टा पहिले
27     ५ घण्टा पहिले
Name: Date, dtype: object

**What does that implies ?**

१५ मिनेट पहिले $\implies$ 15 minutes ago\
१ घण्टा पहिले $\implies$ 1 hour ago\
६ घण्टा पहिले $\implies$ 6 hours ago\
८ घण्टा पहिले $\implies$ 8 hours ago\
४ घण्टा पहिले $\implies$ 4 hours ago\
७ घण्टा पहिले $\implies$ 7 hours ago\
१२ घण्टा पहिले $\implies$ 12 hours ago\
११ घण्टा पहिले $\implies$ 11 hours ago\
३ घण्टा पहिले $\implies$ 3 hours ago\
५ घण्टा पहिले $\implies$ 5 hours ago

These news added in the same date of the data scrapping took place $\implies$ date is **17-08-2023**

In [16]:
# https://github.com/SushilShrestha/pyBSDate#class-based-wrappers

def convert_date_time(date_):

    date_split = date_.split()
    if len(date_split) != 3:
        date_, month_, year_ = date_split[1:]
        month_ = months_dict[month_]
        day_ = int(''.join([number_dict[dt] for dt in list(date_)]))
        year_ = int(''.join([number_dict[yr] for yr in list(year_)]))
        
        return bsdate(year_, month_, day_).addate.strftime('%d-%m-%Y')
    return '17-08-2023'

hmimalkhabar_df['published_date'] = hmimalkhabar_df.Date.apply(convert_date_time)

hmimalkhabar_df.sample(5)

,title,category,authors,Date,content,url,website_name,language,published_date
1194,‘जनयुद्ध दिवस’ मा बिदा दिने निर्णय विरुद्धको र...,समाचार,हिमालखबर,बुधबार २२ चैत २०७९,सर्वोच्च अदालतले ‘जनयुद्ध दिवस’ मा बिदा दिने स...,https://www.himalkhabar.com/news/135184,HimalKhabar,Nepali,05-04-2023
9231,डा. केसी पक्ष र सरकारी टोलीबीचको वार्ता विना न...,स्वास्थ्य,हिमालखबर,शुक्रबार २३ असोज २०७७,चिकित्सा शिक्षा सुधारका अभियन्ता डा. गोविन्द क...,https://www.himalkhabar.com/news/119332,HimalKhabar,Nepali,09-10-2020
9749,हिरासतमा भइरहेका मृत्युबारे छानबिन गर्ने संसदी...,समाचार,हिमालखबर,बिहीबार १ असोज २०७७,"प्रतिनिधिसभाको कानून, न्याय तथा मानवअधिकार समि...",https://www.himalkhabar.com/news/118499,HimalKhabar,Nepali,17-09-2020
20820,संविधान संशोधन विधेयकको विरोधमा तौलिहवामा प्रद...,समाचार,हिमालखबर,बिहीबार २८ पुस २०७३,संविधान संशोधन विधेयकको विरोधमा कपिलवस्तुको तौ...,https://www.himalkhabar.com/news/1771,HimalKhabar,Nepali,12-01-2017
5173,"भारतमा पनि देखियो ओमिक्रोन, दुई जनामा सङ्क्रमण...",कोभिड-१९,हिमालखबर,बिहीबार १६ मंसिर २०७८,कोरोनाभाइरसको नयाँ भेरियन्ट ओमिक्रोन भारतमा पन...,https://www.himalkhabar.com/news/126936,HimalKhabar,Nepali,02-12-2021


## Exploring data

In [17]:
# Number of categories and its counts

print(f'Number of unique categories in hmimalkhabar dataset : {hmimalkhabar_df.category.nunique()}')
hmimalkhabar_df.category.value_counts().head(6)

Number of unique categories in hmimalkhabar dataset : 61


category
समाचार      17781
राजनीति      1008
अर्थ          339
चुनाव         269
कोभिड-१९      263
अदालत         183
Name: count, dtype: int64

## Creating final schema for saving the processed data

In [18]:
print(f'Initial dataset size = {original_shape[0]}\n')
print(f'Number of rows removed = {rows_removed + mo_of_rejected_news}\n')
print(f'Final useful dataset size = {hmimalkhabar_df.shape[0]}\n')

Initial dataset size = 21705

Number of rows removed = 158

Final useful dataset size = 21547



In [19]:
hmimalkhabar_processed = hmimalkhabar_df[['website_name', 'category', 'title', 'authors', 'published_date', 'content', 'language', 'url']].reset_index(drop = True)
print(f'Shape of processed dataset = {hmimalkhabar_processed.shape}\n')
hmimalkhabar_processed.head()

Shape of processed dataset = (21547, 8)



,website_name,category,title,authors,published_date,content,language,url
0,HimalKhabar,समाचार,"बरामद सुनको तौल ६० किलो ७१६ ग्राम कायम, तीन दि...",हिमालखबर,15-08-2023,त्रिभुवन विमानस्थलबाट बाहिरिंदै गर्दा राजस्व अ...,Nepali,https://www.himalkhabar.com/news/137535
1,HimalKhabar,समाचार,सप्तकोशी ब्यारेजका ५६ वटै ढोका खोलिए,हिमालखबर,14-08-2023,प्रहरी नायब उपरीक्षक सिंहले ब्यारेजमा खतराको स...,Nepali,https://www.himalkhabar.com/news/137512
2,HimalKhabar,समाचार,ललिता निवास प्रकरणमा के हो लीलामणि पौडेलको संल...,हिमालखबर,14-08-2023,तत्कालीन मुख्यसचिव लीलामणि पौडेलले सामाजिक समि...,Nepali,https://www.himalkhabar.com/news/137527
3,HimalKhabar,सरकार,"जनमत सरकारमा सहभागी, साह बनिन् मन्त्री",हिमालखबर,14-08-2023,जनमत पार्टी पनि संघीय सरकारमा सहभागी भएको छ भन...,Nepali,https://www.himalkhabar.com/news/137513
4,HimalKhabar,समाचार,जोख्न थालियो विमानस्थलबाट बरामद सुन (तस्वीरहरू),गोपेन राई,14-08-2023,राजस्व अनुसन्धान विभागले बरामद गरेको करीब १०० ...,Nepali,https://www.himalkhabar.com/news/137517


In [20]:
hmimalkhabar_processed.to_json('hmimalkhabar_processed.json')
hmimalkhabar_processed.to_csv('hmimalkhabar_processed.csv', index = False)

In [21]:
hmimalkhabar_processed.shape ==  pd.read_json('hmimalkhabar_processed.json').shape

True

In [22]:
hmimalkhabar_processed.shape ==  pd.read_csv('hmimalkhabar_processed.csv').shape

False

In [23]:
pd.read_csv('hmimalkhabar_processed.csv').shape

(21919, 8)

<div class="alert alert-warning" role="alert">
  <p><strong><u>NOTE</u> :</strong> Indicates CSV have some reliability issue, thus uploading the JSON file</p>
</div>